In [1]:
import pickle as pkl
import numpy as np
import networkx as nx

# 实验结果对比

In [5]:
def readRes(data_name, model_name, metric):
    data_path = "../data/realnets/{}/train_data/SIR_500.pkl_{}_testResults.pkl".format(data_name, model_name)
    with open(data_path, "rb") as f:
        data = pkl.load(f)
    
    dmp = data["dmp_predict"]
    model = data["test_predict"]
    label = data["test_label"]
    
    DMP = []
    MODEL = []
    
    for d, m, l in zip(dmp, model, label):
        if metric == "all":
            d = d.flatten()
            m = m.flatten()
            l = l.flatten()
            
            dl1 = np.mean(np.abs(d-l))
            ml1 = np.mean(np.abs(m-l))
            
            DMP.append(dl1)
            MODEL.append(ml1)
            
        elif metric == "final":
            d = d[-1].flatten()
            m = m[-1].flatten()
            l = l[-1].flatten()
            
            dl1 = np.mean(np.abs(d-l))
            ml1 = np.mean(np.abs(m-l))
            
            DMP.append(dl1)
            MODEL.append(ml1)
            
        elif metric == "final_R":
            d = d[-1, :, -1].flatten()
            m = m[-1, :, -1].flatten()
            l = l[-1, :, -1].flatten()
            
            dl1 = np.mean(np.abs(d-l))
            ml1 = np.mean(np.abs(m-l))
            
            DMP.append(dl1)
            MODEL.append(ml1)
            
    return np.mean(DMP), np.mean(MODEL)

In [12]:
def KL(data_name, model_name):
    data_path = "../data/realnets/{}/train_data/SIR_500.pkl_{}_testResults.pkl".format(data_name, model_name)
    with open(data_path, "rb") as f:
        data = pkl.load(f)
    
    dmp = data["dmp_predict"]
    model = data["test_predict"]
    label = data["test_label"]
    
    DMP = []
    MODEL = []
    
    for d, m, l in zip(dmp, model, label):
        print(d.shape, m.shape, l.shape)
        break

        dl1 = np.mean(np.abs(d-l))
        ml1 = np.mean(np.abs(m-l))

        DMP.append(dl1)
        MODEL.append(ml1)

In [13]:
KL("dolphins", "gnn")

(38, 62, 3) (38, 62, 3) (38, 62, 3)


In [6]:
data_names = ["dolphins",  "fb-food",  "fb-social",  "norwegain",  "openflights",  "top-500",  "us-power"]

In [9]:
for i in range(7):
    dmp, gnn = readRes(data_names[i], model_name="gnn", metric="all")
    print(dmp, gnn)

0.081813276 0.014138435
0.11723867 0.022331337
0.03582426 0.0167742
0.09236728 0.012863914
0.062767625 0.01932762
0.06386378 0.013967746


FileNotFoundError: [Errno 2] No such file or directory: '../data/realnets/us-power/train_data/SIR_500.pkl_gnn_testResults.pkl'

In [8]:
DMP = []
GNN = []
LGNN = []
NEDMP = []
for i in range(7):
    dmp, gnn = readRes(data_names[i], model_name="gnn", metric="all")
    dmp, lgnn = readRes(data_names[i], model_name="gnn", metric="all")
    dmp, nedmp = readRes(data_names[i], model_name="nedmp", metric="all")
    DMP.append(dmp)
    GNN.append(gnn)
    LGNN.append(lgnn)
    NEDMP.append(nedmp)
    
print(DMP)
print(GNN)
print(LGNN)
print(NEDMP)

print(np.mean(DMP))
print(np.mean(GNN))
print(np.mean(LGNN))
print(np.mean(NEDMP))

FileNotFoundError: [Errno 2] No such file or directory: '../data/realnets/fb-food/train_data/SIR_500.pkl_nedmp_testResults.pkl'

In [ ]:
DMP = []
GNN = []
LGNN = []
NEDMP = []
for i in range(7):
    dmp, gnn = readRes(data_names[i], model_name="gnn", metric="final")
    dmp, lgnn = readRes(data_names[i], model_name="lgnn", metric="final")
    dmp, nedmp = readRes(data_names[i], model_name="nedmp", metric="final")
    DMP.append(dmp)
    GNN.append(gnn)
    LGNN.append(lgnn)
    NEDMP.append(nedmp)
    
print(DMP)
print(GNN)
print(LGNN)
print(NEDMP)

print(np.mean(DMP))
print(np.mean(GNN))
print(np.mean(LGNN))
print(np.mean(NEDMP))

In [33]:
for x in NEDMP:
    print("& {:.3f}".format(x), end="          ")

& 0.046          & 0.064          & 0.042          & 0.037          & 0.051          & 0.036          & 0.131          

# Real Nets统计信息

In [84]:
def netsStatic(data_name):
    data_path = "../data/realnets/{}/graph.pkl".format(data_name)
    with open(data_path, "rb") as f:
        g = pkl.load(f)
    
    N = g.number_of_nodes()
    E = g.number_of_edges()
    
    ad = 2*E/N
    ac = nx.average_clustering(g)
    
    adj = nx.adjacency_matrix(g).toarray()
    eigvals = np.real(np.linalg.eigvals(adj))
    
    return N, E, ad, ac, np.max(eigvals)

In [85]:
for i in range(7):
    print(data_names[i])
    N, E, ad, ac, eig = netsStatic(data_names[i])
    print(N,E,ad, ac, 1/eig)

dolphins
62 159 5.129032258064516 0.2589582460550202 0.13901218467687768
fb-food
620 2102 6.780645161290322 0.3308970263553271 0.042227887600146674
fb-social
1899 20296 21.375460768825697 0.08723899580135278 0.029193120697011663
norwegain
1482 4006 5.406207827260459 0.9185760430865096 0.09760837081846596
openflights
2939 15677 10.668254508336169 0.45263386634451047 0.015873334816581475
top-500
500 2980 11.92 0.6174892374001621 0.020801236854151663
us-power
4941 6594 2.66909532483303 0.08010361108159712 0.13363532549148552


In [88]:
g

In [92]:
data_path = "../data/realnets/fb-social/graph.pkl"
with open(data_path, "rb") as f:
    g = pkl.load(f)

In [109]:
with open("../data/synthetic/tree/train_data/SIR_200.pkl", "rb") as f:
    data = pkl.load(f)

In [110]:
data.keys()

dict_keys(['config', 'graph', 'seeds', 'snode2edge', 'tnode2edge', 'edge2tnode', 'nb_matrix', 'node_prob', 'edge_prob', 'node_seed', 'simu_marginal', 'dmp_marginal', 'node2edge'])

In [111]:
g = data["graph"][0]

In [112]:
g.number_of_edges()

24

In [113]:
g.has_edge(0,2)

True

In [114]:
g.has_edge(2,0)

True

In [106]:
data["edge_prob"][0].shape

(20296, 1)

In [102]:
g.edges()

OutEdgeView([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (0, 32), (0, 33), (2, 0), (2, 1), (2, 34), (2, 35), (2, 36), (2, 37), (2, 4), (2, 5), (2, 38), (2, 6), (2, 7), (2, 39), (2, 40), (2, 41), (2, 42), (2, 43), (2, 44), (2, 45), (2, 46), (2, 47), (2, 48), (2, 49), (2, 10), (2, 11), (2, 50), (2, 12), (2, 51), (2, 52), (2, 53), (2, 54), (2, 55), (2, 56), (2, 57), (2, 58), (2, 59), (2, 60), (2, 61), (2, 62), (2, 63), (2, 64), (2, 65), (2, 66), (2, 17), (2, 67), (2, 68), (2, 69), (2, 18), (2, 70), (2, 19), (2, 71), (2, 20), (2, 72), (2, 73), (2, 74), (2, 75), (2, 76), (2, 77), (2, 78), (2, 79), (2, 80), (2, 81), (2, 82), (2, 22), (2, 83), (2, 84), (2, 85), (2, 86), (2, 87), (2, 88), (2, 89), (2, 90), (2, 91), (2, 92), (2, 93), (2, 94), (2, 95), (2, 96), (2, 97), (2,

In [115]:
import networkx as nx

In [116]:
g = nx.Graph()

In [117]:
g.add_edges_from([[0,1], [1, 0], [1,2]])

In [118]:
g.edges()

EdgeView([(0, 1), (1, 2)])

In [119]:
gg = nx.DiGraph(g)

In [120]:
gg.edges()

OutEdgeView([(0, 1), (1, 0), (1, 2), (2, 1)])

In [121]:
with open("../data/realnets/fb-social/graph.pkl", "rb") as f:
    g = pkl.load(f)

In [122]:
g